In [1]:
import config
import mysql.connector
from IPython.display import display, Markdown
import csv
import os

### Helper Functions

In [2]:
def batch_execute_ddl(conn, file):
    cursor = conn.cursor()
    ddl_file = open(file)
    sql = ddl_file.read()
    
    # print(sql)
    # multi = True for multple statements
    # cursor.execute(sql, multi=True)
    for result in cursor.execute(sql, multi=True):
        if result.with_rows:
            print(f"Rows produced: {result.statement}")
            print(result.fetchall())
        else: 
            print(f"number of rows affected by statement {result.statement}: {result.rowcount}")
    
    conn.close()
    
def insert_data(command, data, conn):
    try:
        # Connecting to the Database 
        # print(data)
        cursor = conn.cursor()
        cursor.executemany(command, data) 
        
        conn.commit()
        print("Rows affected: ",cursor.rowcount, "\nData inserted successfully")

    except mysql.connector.Error as error:
        print(error)
        conn.rollback()

        
def fetch(query, conn):
    cursor = conn.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    return result
    # print("Result:", result)
    
# def update(query

def update(query, conn, arg = None, multiflag = False):
    try:
        cursor = conn.cursor()
        if(multiflag == False):
            if (arg != None):
                cursor.execute(query % arg)
            else:
                cursor.execute(query)  
        elif(multiflag):
            if (arg != None):
                cursor.execute(query % arg, multi = True)
            else:
                cursor.execute(query, multi = True)
            
        result = cursor.fetchall()
        # print("Result:", result)
        
        conn.commit()
    except mysql.connector.Error as error:
        print(error)
        conn.rollback()
        
def display(showquery, conn):
    table = fetch(showquery, conn)
    print("{0:20}{1:20}{2:20}".format("branch_name","branch_city","assets"))
    for (branch_name, branch_city, assets) in table:
        print("{0:20}{1:20}{2:<20}".format(branch_name, branch_city, assets))
        
    # fetch(showquery, conn)
    
    
    


In [3]:
conn = mysql.connector.connect(host=config.HOST,
                               user=config.USER, 
                               password=config.PASS)

print(conn.is_connected())

batch_execute_ddl(conn, 'bankdb.sql')

True
number of rows affected by statement drop database bankdb: 6
number of rows affected by statement create database if not exists bankdb: 1
number of rows affected by statement USE bankdb: 0
number of rows affected by statement CREATE TABLE branch (
  branch_name varchar(255) PRIMARY KEY,
  branch_city varchar(255),
  assets decimal (12,2)
): 0
number of rows affected by statement CREATE TABLE customer (
  customer_name varchar(255) PRIMARY KEY,
  customer_street varchar(255),
  customer_city varchar(255)
): 0
number of rows affected by statement CREATE TABLE account (
  account_number varchar(255) PRIMARY KEY,
  branch_name varchar(255),
  balance decimal (12,2) CHECK(balance >= 0),
  FOREIGN KEY (branch_name) REFERENCES branch (branch_name)
): 0
number of rows affected by statement CREATE TABLE depositor (
  customer_name varchar(255),
  account_number varchar(255),
  FOREIGN KEY (account_number) REFERENCES account (account_number),
  FOREIGN KEY (customer_name) REFERENCES custome

In [4]:
path = "activity2_csvs/"
tables = os.listdir("activity2_csvs/") 

keys = [os.path.splitext(table)[0] for table in tables]

d = dict.fromkeys(keys)
# print(d)
for key in keys:
    with open((path+key+'.csv'), newline='') as file:
        reader = csv.reader(file, delimiter=',')
        #ignore the column name
        next(reader)
        
        rows = []
        for row in reader:
            rows.append(tuple(row))
            
        d[key] = rows


customer_insert = """insert into customer
                  (customer_name, customer_street, customer_city)
                  values(%s, %s, %s);"""

branch_insert =  """insert into branch
                 (branch_name, branch_city, assets) 
                 values(%s, %s, %s);"""

account_insert = """insert into account
                    (account_number, branch_name, balance)
                    values(%s, %s, %s);"""

depositor_insert = """insert into depositor
                    (customer_name, account_number)
                    values(%s, %s);"""

loan_insert = """insert into loan
                (loan_number, branch_name, amount)
                values(%s, %s, %s);"""

borrower_insert = """insert into borrower
                    (customer_name, loan_number)
                    values(%s, %s);"""


conn = mysql.connector.connect(host=config.HOST,
                               user=config.USER, 
                               password=config.PASS,
                               database=config.MYDB)

insert_data(branch_insert, d['branch'], conn)
insert_data(customer_insert, d['customer'], conn)
insert_data(account_insert, d['account'], conn)
insert_data(depositor_insert, d['depositor'], conn)
insert_data(loan_insert, d['loan'], conn)
insert_data(borrower_insert, d['borrower'], conn)

query = "select sum(assets*0.125) from branch where (assets >= 3000000);"

result = fetch(query, conn)[0][0]

query = "update branch set assets = assets*(1-0.125) where (assets >= 3000000);"

update(query, conn)
# conn.cursor().execute(query)
# conn.commit()

query = """update branch,  
           (select count(*) as total from branch where assets < 3000000) T2
           set assets = ( %s / T2.total ) + assets
           where (assets < 3000000)
           """
# conn.cursor().execute(query % result)
# conn.commit()
update(query, conn, result)

conn.close()

# showquery = "select * from branch order by assets desc;"

# display(showquery, conn)

Rows affected:  9 
Data inserted successfully
Rows affected:  15 
Data inserted successfully
Rows affected:  9 
Data inserted successfully
Rows affected:  10 
Data inserted successfully
Rows affected:  9 
Data inserted successfully
Rows affected:  10 
Data inserted successfully


# Activity 3
 Imagine that the bank has received a portion of a government bailout totaling 10 million dollars. One requirement for
banks receiving this bailout money is that they remain their current size (i.e., large branches are branches with assets
\>= 3 million dollars and small branches have assets < 3 million dollars).The CEO of the bank, Nate Richmond, would
like to divide this bailout money equally among the small branches at the bank.
In a Jupyter Notebook, write code in Python, Java or C (or a language that works in Jupyter) to help DBA Christopher
Hemsworth complete the following:

In [5]:
conn = mysql.connector.connect(host=config.HOST,
                               user=config.USER, 
                               password=config.PASS,
                               database=config.MYDB)

## a
Execute a single query to return and print out the total assets owned by the bank ( i.e., a cumulative total of the assets at all branches)

In [6]:
query = "SELECT SUM(assets) FROM branch;"

result = fetch(query, conn)[0][0]
print("Total assets owned by the bank: ", result)

Total assets owned by the bank:  24600479.98


## b
Create a trigger named maintain_branch_size on your branch table that ensures that a small
branch does not become a large branch (i.e., that its total assets never gets to 3 million dollars). Whenever an
update to the assets of a small branch is attempted that would cause it to become a large branch, your trigger
should store this excess money in a table (e.g. bailout_cache) to be later divided equally among the existing
large branches, capping the small branch at 2,999,999 in total assets.

In [7]:
# DECLARE  bailout FLOAT(12,2) DEFAULT 0;
# INSERT bailout_cache (amount) VALUES (NEW.assets - 2999999);
#AND NEW.assets > 3000000 

                 # IF NEW.assets > 3000000 THEN
                 #     END IF;

table = """create table if not exists bailout_cache(amount float(12,2));"""

update(table,conn)

trigger = """create trigger maintain_branch_size before update on branch
            for each row
            begin
            DECLARE m float default 2999999.99;
            if (new.assets > m and old.assets <= m) then
            insert into bailout_cache values(new.assets-m);
            set new.assets = m;
            end if;
            end;
            """

update(trigger,conn)

## c
Execute a query that attempts to divide the bailout money among the small branches as a test of
your trigger.

In [8]:
query = """update branch,  
           (select count(*) as total from branch where assets < 3000000) small_branches
           set assets = ( 10000000 / small_branches.total ) + assets
           where (assets < 3000000)
           """

print("BEFORE:")
display("select * FROM branch order by assets desc;", conn)

print("\n")

update(query, conn)


print("AFTER:")
display("select * from branch order by assets desc;", conn)

BEFORE:
branch_name         branch_city         assets              
Round Hill          Horseneck           7000000.00          
Brighton            Brooklyn            6125000.00          
North Town          Rye                 3237500.00          
Redwood             Palo Alto           2489583.33          
Perryridge          Horseneck           2089583.33          
Downtown            Brooklyn            1289583.33          
Central             Rye                 789863.33           
Mianus              Horseneck           789783.33           
Pownal              Bennington          789583.33           


AFTER:
branch_name         branch_city         assets              
Round Hill          Horseneck           7000000.00          
Brighton            Brooklyn            6125000.00          
North Town          Rye                 3237500.00          
Perryridge          Horseneck           3000000.00          
Redwood             Palo Alto           3000000.00          
Downtow

## d
Execute a query that divides any excess money from the bailout (money in your bailout_cache) among the large branches

## e 
Execute a single query to return and print out the total assets owned by the bank ( i.e., a cumulative
total of the assets at all branches) after all changes above has been made.

In [10]:
cumulative = fetch("select sum(assets) from branch", conn)[0][0]
print("Total assets owned by the bank: ", cumulative)

Total assets owned by the bank:  34600480.00


In [11]:
conn.close()